###Pip installs, mount drive, set paths etc.

In [ ]:
!pip install mne

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import mne
import math
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.metrics import confusion_matrix as cm
import seaborn as sns
from collections import Counter
import sys

     |████████████████████████████████| 6.9MB 4.3MB/s 


In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)

#Set paths
train_path = 'drive/MyDrive/BA thesis/Data/train'
train_labels_path = 'drive/MyDrive/BA thesis/Data/TrainLabels.csv'
test_path = 'drive/MyDrive/BA thesis/Data/test'
test_labels_path = 'drive/MyDrive/BA thesis/Data/TestLabels.csv'
channel_location_path = 'drive/MyDrive/BA thesis/Data/ChannelsLocation.csv'

Mounted at /content/drive


###Loading data

In [ ]:
# Loading and reading .csv files
train_directory = os.fsencode('drive/MyDrive/BA thesis/Data/train')
test_directory = os.fsencode('drive/MyDrive/BA thesis/Data/test')

train_labels = pd.read_csv('drive/MyDrive/BA thesis/Data/TrainLabels.csv')['Prediction']
test_labels = pd.read_csv('drive/MyDrive/BA thesis/Data/TestLabels.csv')['Prediction']

In [ ]:
for file in os.listdir(train_directory):   
    print("filename: " + str(file))

filename: b'Data_S02_Sess01.csv'
filename: b'Data_S02_Sess02.csv'
filename: b'Data_S02_Sess03.csv'
filename: b'Data_S02_Sess04.csv'
filename: b'Data_S02_Sess05.csv'
filename: b'Data_S06_Sess01.csv'
filename: b'Data_S06_Sess02.csv'
filename: b'Data_S06_Sess03.csv'
filename: b'Data_S06_Sess04.csv'
filename: b'Data_S06_Sess05.csv'
filename: b'Data_S07_Sess01.csv'
filename: b'Data_S07_Sess02.csv'
filename: b'Data_S07_Sess03.csv'
filename: b'Data_S07_Sess04.csv'
filename: b'Data_S07_Sess05.csv'
filename: b'Data_S11_Sess01.csv'
filename: b'Data_S11_Sess02.csv'
filename: b'Data_S11_Sess03.csv'
filename: b'Data_S11_Sess04.csv'
filename: b'Data_S11_Sess05.csv'
filename: b'Data_S12_Sess01.csv'
filename: b'Data_S12_Sess02.csv'
filename: b'Data_S12_Sess03.csv'
filename: b'Data_S12_Sess04.csv'
filename: b'Data_S12_Sess05.csv'
filename: b'Data_S13_Sess01.csv'
filename: b'Data_S13_Sess02.csv'
filename: b'Data_S13_Sess03.csv'
filename: b'Data_S13_Sess04.csv'
filename: b'Data_S13_Sess05.csv'
filename: 

In [ ]:
train_labels

0       1
1       1
2       0
3       0
4       1
       ..
5435    1
5436    0
5437    0
5438    0
5439    1
Name: Prediction, Length: 5440, dtype: int64

In [ ]:
channels = pd.read_csv(channel_location_path)['Labels']
ch_types = ['eeg']*len(channels) + ['eog'] + ['stim']
channels = channels.to_list()
channels.append('EOG')
channels.append('Feedback')

In [ ]:
# mne
info = mne.create_info(ch_names = channels, ch_types = ch_types, sfreq = 200)
iir_param = dict(order=4, ftype = 'butter', f_pass = [1,20], sfreq = 200, btype = 'bandpass')

In [ ]:
#channel_id = [10,28,44,46,48,51]
#Fz, Cz, P3, Pz, P4, PO7

def make_mne_epoch(directory,label):
    count = 0

    for file in os.listdir(directory):   
        print("filename: " + str(file) + " - filecount: " + str(count))

        filename = os.fsdecode(file)
        filename = os.path.join(directory.decode('utf-8'), filename)
        my_data_raw = pd.read_csv(filename)

        my_data_raw = my_data_raw.drop(['Time'],axis=1)
        my_data = np.array(my_data_raw)
        my_data = my_data.T

        MNE_raw = mne.io.RawArray(my_data,info,verbose='WARNING')

        Event_list = mne.find_events(MNE_raw, stim_channel = 'Feedback')
        MNE_raw.filter(1, 20,method='iir',iir_params=iir_param)

        if count == 0:
            MNE_epochs = mne.Epochs(MNE_raw, events = Event_list, tmin=-0.25, tmax=1, preload = True,verbose='WARNING')
        else:
            MNE_epochs_new = mne.Epochs(MNE_raw, events = Event_list, tmin=-0.25, tmax=1, preload = True, verbose='WARNING')
            MNE_epochs = mne.concatenate_epochs([MNE_epochs,MNE_epochs_new])

        count += 1

    MNE_epochs.apply_baseline((None,0))
    MNE_epochs.resample(sfreq=100)

    MNE_epochs.events[:,2] = label
    event_dict = {'Positive': 1, 'Negative': 0}
    MNE_epochs.event_id = event_dict

    return MNE_epochs

In [ ]:
train_epochs = make_mne_epoch(train_directory,train_labels)

filename: b'Data_S02_Sess01.csv' - filecount: 0
60 events found
Event IDs: [1]
filename: b'Data_S02_Sess02.csv' - filecount: 1
60 events found
Event IDs: [1]
filename: b'Data_S02_Sess03.csv' - filecount: 2
60 events found
Event IDs: [1]
filename: b'Data_S02_Sess04.csv' - filecount: 3
60 events found
Event IDs: [1]
filename: b'Data_S02_Sess05.csv' - filecount: 4
100 events found
Event IDs: [1]
filename: b'Data_S06_Sess01.csv' - filecount: 5
60 events found
Event IDs: [1]
filename: b'Data_S06_Sess02.csv' - filecount: 6
60 events found
Event IDs: [1]
filename: b'Data_S06_Sess03.csv' - filecount: 7
60 events found
Event IDs: [1]
filename: b'Data_S06_Sess04.csv' - filecount: 8
60 events found
Event IDs: [1]
filename: b'Data_S06_Sess05.csv' - filecount: 9
100 events found
Event IDs: [1]
filename: b'Data_S07_Sess01.csv' - filecount: 10
60 events found
Event IDs: [1]
filename: b'Data_S07_Sess02.csv' - filecount: 11
60 events found
Event IDs: [1]
filename: b'Data_S07_Sess03.csv' - filecount: 12

In [ ]:
#Saving training data epochs as fif on drive
train_epochs.save('training-epo.fif')

In [ ]:
test_epochs = make_mne_epoch(test_directory,test_labels)

filename: b'Data_S01_Sess01.csv' - filecount: 0
60 events found
Event IDs: [1]
filename: b'Data_S01_Sess02.csv' - filecount: 1
60 events found
Event IDs: [1]
filename: b'Data_S01_Sess03.csv' - filecount: 2
60 events found
Event IDs: [1]
filename: b'Data_S01_Sess04.csv' - filecount: 3
60 events found
Event IDs: [1]
filename: b'Data_S01_Sess05.csv' - filecount: 4
100 events found
Event IDs: [1]
filename: b'Data_S03_Sess01.csv' - filecount: 5
60 events found
Event IDs: [1]
filename: b'Data_S03_Sess02.csv' - filecount: 6
60 events found
Event IDs: [1]
filename: b'Data_S03_Sess03.csv' - filecount: 7
60 events found
Event IDs: [1]
filename: b'Data_S03_Sess04.csv' - filecount: 8
60 events found
Event IDs: [1]
filename: b'Data_S03_Sess05.csv' - filecount: 9
100 events found
Event IDs: [1]
filename: b'Data_S04_Sess01.csv' - filecount: 10
60 events found
Event IDs: [1]
filename: b'Data_S04_Sess02.csv' - filecount: 11
60 events found
Event IDs: [1]
filename: b'Data_S04_Sess03.csv' - filecount: 12

In [ ]:
test_epochs.save('test-epo.fif')